In [182]:
#imports 

import numpy as np
import os, tempfile
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing.image import load_img

from tensorflow.keras.models import Sequential

from sklearn.dummy import DummyClassifier

import time
import scipy
from scipy import ndimage
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from PIL import Image
from tensorflow.keras.preprocessing import image_dataset_from_directory

import joblib

## Image Load In

In [183]:
!ls

Data
Peter.ipynb
PeterNotebook.ipynb
README.md
Yuhkai_notebook_scratch.ipynb


In [184]:
!cd ..

In [185]:
!ls

Data
Peter.ipynb
PeterNotebook.ipynb
README.md
Yuhkai_notebook_scratch.ipynb


In [186]:
!ls

Data
Peter.ipynb
PeterNotebook.ipynb
README.md
Yuhkai_notebook_scratch.ipynb


In [187]:
# load in the train data

#starting from main project directory

train_normal_dir = "./data/chest_xray/train/NORMAL"
train_pneumonia_dir = "./data/chest_xray/train/PNEUMONIA"

imgs_train_normal = [file for file in os.listdir(train_normal_dir) if file.endswith('.jpeg')]
imgs_train_pneumonia = [file for file in os.listdir(train_pneumonia_dir) if file.endswith('.jpeg')]

In [188]:
test_normal_dir = "./data/chest_xray/test/NORMAL"
test_pneumonia_dir = "./data/chest_xray/test/PNEUMONIA"

imgs_test_normal = [file for file in os.listdir(test_normal_dir) if file.endswith('.jpeg')]
imgs_test_pneumonia = [file for file in os.listdir(test_pneumonia_dir) if file.endswith('.jpeg')]

In [189]:
len(imgs_train_normal)

1341

In [190]:
len(imgs_train_pneumonia)

3875

In [191]:
imgs_train_normal[0]

'IM-0115-0001.jpeg'

In [192]:
imgs_train_pneumonia[0]

'person1000_bacteria_2931.jpeg'

In [193]:
len(imgs_test_normal)

234

In [194]:
len(imgs_test_pneumonia)

390

In [195]:
num_train = len(imgs_train_normal) + len(imgs_train_pneumonia)
num_test = len(imgs_test_normal) + len(imgs_test_pneumonia)
num_train, num_test

(5216, 624)

In [196]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

In [221]:
#rescale images to 64 by 64
#create validation set as 20% of train set

train_folder = "./data/chest_xray/train"
train_gen = train_datagen.flow_from_directory(
        train_folder, 
        target_size=(64, 64), 
       
        class_mode='binary',
        subset='training',
        batch_size= num_train)

validation_gen = train_datagen.flow_from_directory(
        train_folder, 
        target_size=(64, 64), 
       
        class_mode='binary',
        subset='validation',
        batch_size= num_train)

test_folder = "./data/chest_xray/test"
test_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
        test_folder, 
        target_size=(64, 64), 
        class_mode='binary', 
        batch_size= num_test)

Found 4173 images belonging to 2 classes.
Found 1043 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [222]:
train_images, train_labels = next(train_gen)
val_images, val_labels = next(validation_gen)
test_images, test_labels = next(test_gen)

In [223]:
print(train_images.shape)

(4173, 64, 64, 3)


In [224]:
train_images[0]

array([[[0.03529412, 0.03529412, 0.03529412],
        [0.21568629, 0.21568629, 0.21568629],
        [0.20000002, 0.20000002, 0.20000002],
        ...,
        [0.1254902 , 0.1254902 , 0.1254902 ],
        [0.10196079, 0.10196079, 0.10196079],
        [0.09411766, 0.09411766, 0.09411766]],

       [[0.2509804 , 0.2509804 , 0.2509804 ],
        [0.21176472, 0.21176472, 0.21176472],
        [0.10196079, 0.10196079, 0.10196079],
        ...,
        [0.14117648, 0.14117648, 0.14117648],
        [0.10588236, 0.10588236, 0.10588236],
        [0.09803922, 0.09803922, 0.09803922]],

       [[0.12156864, 0.12156864, 0.12156864],
        [0.08627451, 0.08627451, 0.08627451],
        [0.12941177, 0.12941177, 0.12941177],
        ...,
        [0.1764706 , 0.1764706 , 0.1764706 ],
        [0.10980393, 0.10980393, 0.10980393],
        [0.10588236, 0.10588236, 0.10588236]],

       ...,

       [[0.09411766, 0.09411766, 0.09411766],
        [0.05882353, 0.05882353, 0.05882353],
        [0.05490196, 0

In [234]:
val_images[0]

array([[[0.83921576, 0.83921576, 0.83921576],
        [0.8588236 , 0.8588236 , 0.8588236 ],
        [0.8705883 , 0.8705883 , 0.8705883 ],
        ...,
        [0.7960785 , 0.7960785 , 0.7960785 ],
        [0.80392164, 0.80392164, 0.80392164],
        [0.7803922 , 0.7803922 , 0.7803922 ]],

       [[0.8470589 , 0.8470589 , 0.8470589 ],
        [0.86274517, 0.86274517, 0.86274517],
        [0.86274517, 0.86274517, 0.86274517],
        ...,
        [0.6745098 , 0.6745098 , 0.6745098 ],
        [0.8078432 , 0.8078432 , 0.8078432 ],
        [0.80392164, 0.80392164, 0.80392164]],

       [[0.8431373 , 0.8431373 , 0.8431373 ],
        [0.8745099 , 0.8745099 , 0.8745099 ],
        [0.87843144, 0.87843144, 0.87843144],
        ...,
        [0.6666667 , 0.6666667 , 0.6666667 ],
        [0.75294125, 0.75294125, 0.75294125],
        [0.81568635, 0.81568635, 0.81568635]],

       ...,

       [[0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        [0.        , 0

In [236]:
#(image height, image width, color channels)
train_images[0].shape

(64, 64, 3)

In [241]:
#preserving shape for VGG19


## VGG19

In [176]:
from keras.applications import VGG19
from keras import models
from keras import layers
from keras import optimizers


cnn_base = VGG19(weights='imagenet', 
                 include_top=False, 
                 input_shape=(64, 64, 3))

In [ ]:
transfer_model = models.Sequential()
transfer_model.add(cnn_base)

In [177]:
transfer_model = models.Sequential()
transfer_model.add(cnn_base)
transfer_model.add(layers.Flatten())
transfer_model.add(layers.Dense(128, activation='relu'))
transfer_model.add(layers.Dense(1, activation='sigmoid'))

In [178]:
cnn_base.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 64, 64, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0     

In [ ]:
transfer_model.compile(loss='binary_crossentropy',
                optimizer='RMSprop',
                metrics=['accuracy', tf.keras.metrics.Precision(),  tf.keras.metrics.Recall()])

In [271]:
print(train_images.shape)

(4173, 64, 64, 3)


In [274]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping 
from tensorflow.keras.applications import VGG16, VGG19

In [278]:
stop = EarlyStopping(monitor='loss', mode='min', patience=3)
callbacks = [stop]

In [ ]:
transfer_model.fit(train_images, train_labels, epochs=10, batch_size=30, callbacks = [stop])

Epoch 1/10
140/140 [==============================] - 181s 1s/step - loss: 0.8163 - accuracy: 0.7376 - precision_7: 0.7420
Epoch 2/10
 48/140 [=========>....................] - ETA: 2:01 - loss: 0.6010 - accuracy: 0.7208 - precision_7: 0.7208